In [499]:
# Importing necessary libraries
import gzip
import shutil
import os
import pandas as pd
from pandas import json_normalize
import json
import sqlite3

# Data Wrangling 

## Receipts

In [500]:
df_receipts = pd.read_json('data_gzip/receipts.json.gz', lines=True, compression='gzip')

In [501]:
df_receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [502]:
df_receipts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id                      1119 non-null   object 
 1   bonusPointsEarned        544 non-null    float64
 2   bonusPointsEarnedReason  544 non-null    object 
 3   createDate               1119 non-null   object 
 4   dateScanned              1119 non-null   object 
 5   finishedDate             568 non-null    object 
 6   modifyDate               1119 non-null   object 
 7   pointsAwardedDate        537 non-null    object 
 8   pointsEarned             609 non-null    float64
 9   purchaseDate             671 non-null    object 
 10  purchasedItemCount       635 non-null    float64
 11  rewardsReceiptItemList   679 non-null    object 
 12  rewardsReceiptStatus     1119 non-null   object 
 13  totalSpent               684 non-null    float64
 14  userId                  

In [503]:
df_receipts.shape

(1119, 15)

**Cleaning _id and all Date field so that they're are easy to analyse and process. Converting all timestrape to DateTime since all date-based manipulations can be performed directly in your SQL queries, even without having separated date components as individual columns in your database**

In [504]:
df_receipts['_id'] = df_receipts['_id'].apply(lambda x: x.get('$oid') if isinstance(x, dict) else None)
df_receipts.rename(columns={'_id': 'receipts_id'}, inplace=True)

# Similar checks for date fields, converting them to datetime only if they are dictionaries
df_receipts['createDate'] = pd.to_datetime(df_receipts['createDate'].apply(lambda x: x.get('$date') if isinstance(x, dict) else None), unit='ms', errors='coerce')
df_receipts['dateScanned'] = pd.to_datetime(df_receipts['dateScanned'].apply(lambda x: x.get('$date') if isinstance(x, dict) else None), unit='ms', errors='coerce')
df_receipts['finishedDate'] = pd.to_datetime(df_receipts['finishedDate'].apply(lambda x: x.get('$date') if isinstance(x, dict) else None), unit='ms', errors='coerce')
df_receipts['modifyDate'] = pd.to_datetime(df_receipts['modifyDate'].apply(lambda x: x.get('$date') if isinstance(x, dict) else None), unit='ms', errors='coerce')
df_receipts['pointsAwardedDate'] = pd.to_datetime(df_receipts['pointsAwardedDate'].apply(lambda x: x.get('$date') if isinstance(x, dict) else None), unit='ms', errors='coerce')
df_receipts['purchaseDate'] = pd.to_datetime(df_receipts['purchaseDate'].apply(lambda x: x.get('$date') if isinstance(x, dict) else None), unit='ms', errors='coerce')

In [505]:
df_receipts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   receipts_id              1119 non-null   object        
 1   bonusPointsEarned        544 non-null    float64       
 2   bonusPointsEarnedReason  544 non-null    object        
 3   createDate               1119 non-null   datetime64[ns]
 4   dateScanned              1119 non-null   datetime64[ns]
 5   finishedDate             568 non-null    datetime64[ns]
 6   modifyDate               1119 non-null   datetime64[ns]
 7   pointsAwardedDate        537 non-null    datetime64[ns]
 8   pointsEarned             609 non-null    float64       
 9   purchaseDate             671 non-null    datetime64[ns]
 10  purchasedItemCount       635 non-null    float64       
 11  rewardsReceiptItemList   679 non-null    object        
 12  rewardsReceiptStatus     1119 non-

Checking for duplicates for all rows in `df_receipts` is not feasible since it contains a column with unhashable types. The `duplicated()` method works by trying to hash the rows to find duplicates, but dictionaries and lists are mutable and hence unhashable.
Therefore, need to check for duplicates excluding the column with nested dictionaries, that is, `rewardsReceiptItemList`. And this exclusion can be justified from business point of view as well: Row with same `rewardsReceiptItemList` not necessarily mean they are duplicates if they have different `_id`. And this goes for other columns as well, as long as `_id` is unique, it is not duplicate.

In [506]:
duplicate_count = df_receipts['receipts_id'].duplicated().sum()
print (duplicate_count)

0


In [507]:
df_receipts.isnull().sum()

receipts_id                  0
bonusPointsEarned          575
bonusPointsEarnedReason    575
createDate                   0
dateScanned                  0
finishedDate               551
modifyDate                   0
pointsAwardedDate          582
pointsEarned               510
purchaseDate               448
purchasedItemCount         484
rewardsReceiptItemList     440
rewardsReceiptStatus         0
totalSpent                 435
userId                       0
dtype: int64

In our `df_receipts` dataset, we encounter missing values in several columns such as `bonusPointsEarned`, `finishedDate`, and `totalSpent`. Addressing these is crucial for data integrity and analysis readiness. 

For numerical fields like `bonusPointsEarned`, imputation with median or mean values is a viable strategy to maintain data distribution. Categorical fields may be filled with the most common value (mode) or a placeholder like `'Unknown'`. Where applicable, business logic can be applied to calculate missing values, enhancing data accuracy.

Columns with extensive missing data that are non-critical can be considered for removal to simplify the dataset. Optionally, flagging missing values can be informative, particularly for modeling purposes.

The approach to handling missing data should balance completeness with quality, ensuring any adjustments are well-documented to clarify their impact on analyses or model outcomes.

### Normalizing the receipts Data

The receipts dataset contains a nested structure within rewardsReceiptItemList. Performing normalization of this into a separate DataFrame since, this will be easier to maintain, have better query performance and better database design.

In [508]:
# Will skip over empty lists since, there are no items to create rows from, 
# no rows will be added to the output DataFrame for these particular records. 
df_receipt_items_list = json_normalize(data=df_receipts.to_dict(orient="records"), record_path='rewardsReceiptItemList', 
                                    meta='receipts_id', 
                                    record_prefix='item_',
                                    errors='ignore')

df_receipts = df_receipts.drop(columns=['rewardsReceiptItemList'])

In [509]:
df_receipt_items_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6941 entries, 0 to 6940
Data columns (total 35 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   item_barcode                             3090 non-null   object 
 1   item_description                         6560 non-null   object 
 2   item_finalPrice                          6767 non-null   object 
 3   item_itemPrice                           6767 non-null   object 
 4   item_needsFetchReview                    813 non-null    object 
 5   item_partnerItemId                       6941 non-null   object 
 6   item_preventTargetGapPoints              358 non-null    object 
 7   item_quantityPurchased                   6767 non-null   float64
 8   item_userFlaggedBarcode                  337 non-null    object 
 9   item_userFlaggedNewItem                  323 non-null    object 
 10  item_userFlaggedPrice                    299 non

In [510]:
df_receipt_items_list.shape

(6941, 35)

## Brands

In [511]:
df_brands = pd.read_json('data_gzip/brands.json.gz', lines=True, compression='gzip')

In [512]:
df_brands.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


In [513]:
df_brands.shape

(1167, 8)

In [514]:
df_brands.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   _id           1167 non-null   object 
 1   barcode       1167 non-null   int64  
 2   category      1012 non-null   object 
 3   categoryCode  517 non-null    object 
 4   cpg           1167 non-null   object 
 5   name          1167 non-null   object 
 6   topBrand      555 non-null    float64
 7   brandCode     933 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 73.1+ KB


In [515]:
df_brands['_id'] = df_brands['_id'].apply(lambda x: x.get('$oid') if isinstance(x, dict) else None)
df_brands.rename(columns={'_id': 'brand_id'}, inplace=True)

In [516]:
df_brands.head()

,brand_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


In [517]:
duplicate_count_brands = df_brands['brand_id'].duplicated().sum()
print (duplicate_count_brands)

0


In [518]:
df_brands.shape

(1167, 8)

In [519]:
df_brands.isnull().sum()

brand_id          0
barcode           0
category        155
categoryCode    650
cpg               0
name              0
topBrand        612
brandCode       234
dtype: int64

Addressing missing values in our `df_brands` DataFrame, with gaps in `category`, `categoryCode`, `topBrand`, and `brandCode`, is pivotal for ensuring data analysis integrity.

Missing values in `topBrand` could pragmatically be treated as `False`, assuming unmarked brands aren't top brands, pending validation. The `brandCode` column, essential for detailed analysis or data linkage, requires careful consideration for imputation or record exclusion. Decisions on handling these missing values—imputation, exclusion, or acceptance—are guided by the dataset's context and our analysis goals, striving for a balance that maintains data integrity while enabling meaningful insights.

### Brands table has nested `cpg` and to have better atomicity and scalablibilty it is best to separate `cpg` into different table and connecting it with `brands` table with foreign key `cpg_id`

In [520]:
df_brands['cpg_id'] = df_brands['cpg'].apply(lambda x: x['$id']['$oid'])
df_brands['cpg_ref'] = df_brands['cpg'].apply(lambda x: x['$ref'])

df_cpg = df_brands[['cpg_id', 'cpg_ref']].drop_duplicates().reset_index(drop=True)

df_brands = df_brands.drop(columns=['cpg'])
df_brands = df_brands.drop(columns=['cpg_ref'])

In [521]:
df_cpg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   cpg_id   204 non-null    object
 1   cpg_ref  204 non-null    object
dtypes: object(2)
memory usage: 3.3+ KB


In [522]:
df_cpg.head()

,cpg_id,cpg_ref
0,601ac114be37ce2ead437550,Cogs
1,5332f5fbe4b03c9a25efd0ba,Cogs
2,601ac142be37ce2ead437559,Cogs
3,5332fa12e4b03c9a25efd1e7,Cogs
4,559c2234e4b06aca36af13c6,Cogs


## Users

**For `users.json.gz`, the result after extraction was corrupted so had to extra processing before could use it in DataFrames. I have kept `users.json` in `src/` folder for reference. 
Error for unprocessed file: `ValueError: Expected object or value` and verified the issue by opening extracted json, it included some random hash both in begining and end of the document. (maybe related to MangoDB)**

In [523]:
with gzip.open('data_gzip/users.json.gz', 'rb') as gz_file:
    with open('users.json', 'wb') as output_file:
        shutil.copyfileobj(gz_file, output_file)

In [524]:
def find_json_start(line):
    """
    Attempt to find the index where a valid JSON object starts.
    """
    for i in range(len(line)):
        try:
            json.loads(line[i:])
            return i
        except json.JSONDecodeError:
            continue
    return None

def clean_json_line(line):
    """
    Try to clean the line by finding where the JSON object starts.
    """
    json_start = find_json_start(line)
    if json_start is not None:
        try:
            return json.loads(line[json_start:])
        except json.JSONDecodeError:
            return None
    else:
        return None

def de_corrupt_json_file(input_file_path, output_file_path):
    """
    Read a potentially corrupted JSON file, clean each line, and write the cleaned data to a new file.
    """
    with open(input_file_path, 'r', encoding='utf-8') as file, open(output_file_path, 'w', encoding='utf-8') as outfile:
        for line in file:
            cleaned_line = clean_json_line(line)
            if cleaned_line is not None:
                json.dump(cleaned_line, outfile)
                outfile.write('\n')

In [525]:
de_corrupt_json_file('users.json', 'processed_users.json')

In [526]:
df_users = pd.read_json('processed_users.json', lines = True)

In [527]:
df_users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [528]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   _id           495 non-null    object
 1   active        495 non-null    bool  
 2   createdDate   495 non-null    object
 3   lastLogin     433 non-null    object
 4   role          495 non-null    object
 5   signUpSource  447 non-null    object
 6   state         439 non-null    object
dtypes: bool(1), object(6)
memory usage: 23.8+ KB


Flattening the nested JSON

In [529]:
df_users['_id'] = df_users['_id'].apply(lambda x: x['$oid'] if isinstance(x, dict) else x)
df_users.rename(columns={'_id': 'user_id'}, inplace=True)

# Convert 'createdDate' and 'lastLogin' to datetime format, handling errors gracefully
df_users['createdDate'] = pd.to_datetime(df_users['createdDate'].apply(lambda x: x.get('$date') if isinstance(x, dict) else None), unit='ms', errors='coerce')
df_users['lastLogin'] = pd.to_datetime(df_users['lastLogin'].apply(lambda x: x.get('$date') if isinstance(x, dict) else None), unit='ms', errors='coerce')



In [530]:
df_users.head()

,user_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30.554,2021-01-03 15:25:30.597,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI


In [531]:
df_users.shape

(495, 7)

In [532]:
duplicates_users = df_users.duplicated().sum()
print(duplicates_users)

283


In [533]:
# Dropping the duplicates
df_users = df_users.drop_duplicates()
df_users.reset_index(drop=True, inplace=True)

In [534]:
df_users.shape

(212, 7)

In [535]:
df_users.isnull().sum()

user_id          0
active           0
createdDate      0
lastLogin       40
role             0
signUpSource     5
state            6
dtype: int64

In [536]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212 entries, 0 to 211
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   user_id       212 non-null    object        
 1   active        212 non-null    bool          
 2   createdDate   212 non-null    datetime64[ns]
 3   lastLogin     172 non-null    datetime64[ns]
 4   role          212 non-null    object        
 5   signUpSource  207 non-null    object        
 6   state         206 non-null    object        
dtypes: bool(1), datetime64[ns](2), object(4)
memory usage: 10.3+ KB


There are not as many missing data in Users table as compared to other two tables. 
Missing data in `lastLogin` can be dealt in following ways:
1. Fill missing values with a placeholder date far in the past to distinguish inactive users.
2. Exclude these rows from analyses where last login date is crucial.

#### We have successfully transformed our data into structured DataFrames, adhering to best practices in database design. This structured format not only facilitates efficient storage in relational databases but also streamlines the process of performing in-depth analysis and making informed business decisions. 
#### By organizing our data into well-defined DataFrames, we ensure that it is optimally prepared for loading into SQL databases, enabling us to leverage the full power of relational database management systems for querying, analysis, and reporting. This strategic approach to data structuring underscores our commitment to data integrity, accessibility, and analytical readiness, positioning us to extract valuable insights that drive strategic business outcomes. 
#### Moving on to writing query portion to answer question!

# Write a query that directly answers a predetermined question from a business stakeholder.
1. When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
2. When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

**I'll be using SQLite for creating the database and quering it.**

In [537]:
# Connecting to SQLite database
conn = sqlite3.connect('data_rd.db')

# Creating a cursor object using the connection
cursor = conn.cursor()

## Creating SQLite DB

In [538]:
# Create tables
cursor.execute('''
CREATE TABLE IF NOT EXISTS users (
    user_id TEXT PRIMARY KEY,
    active BOOLEAN,
    createdDate DATETIME,
    lastLogin DATETIME,
    role TEXT,
    signUpSource TEXT,
    state TEXT
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS receipts (
    receipts_id TEXT PRIMARY KEY,
    bonusPointsEarned REAL,
    bonusPointsEarnedReason TEXT,
    createDate DATETIME,
    dateScanned DATETIME,
    finishedDate DATETIME,
    modifyDate DATETIME,
    pointsAwardedDate DATETIME,
    pointsEarned REAL,
    purchaseDate DATETIME,
    purchasedItemCount REAL,
    rewardsReceiptStatus TEXT,
    totalSpent REAL,
    userId TEXT,
    FOREIGN KEY (userId) REFERENCES users(user_id)
);
''')

# Adding a simple item_id for better data management
cursor.execute('''
CREATE TABLE IF NOT EXISTS receipt_items (
    item_id INTEGER PRIMARY KEY AUTOINCREMENT,
    item_barcode TEXT,
    item_description TEXT,
    item_finalPrice TEXT,
    item_itemPrice TEXT,
    item_needsFetchReview TEXT,
    item_partnerItemId TEXT,
    item_preventTargetGapPoints TEXT,
    item_quantityPurchased REAL,
    item_userFlaggedBarcode TEXT,
    item_userFlaggedNewItem TEXT,
    item_userFlaggedPrice TEXT,
    item_userFlaggedQuantity REAL,
    item_needsFetchReviewReason TEXT,
    item_pointsNotAwardedReason TEXT,
    item_pointsPayerId TEXT,
    item_rewardsGroup TEXT,
    item_rewardsProductPartnerId TEXT,
    item_userFlaggedDescription TEXT,
    item_originalMetaBriteBarcode TEXT,
    item_originalMetaBriteDescription TEXT,
    item_brandCode TEXT,
    item_competitorRewardsGroup TEXT,
    item_discountedItemPrice TEXT,
    item_originalReceiptItemText TEXT,
    item_itemNumber TEXT,
    item_originalMetaBriteQuantityPurchased REAL,
    item_pointsEarned TEXT,
    item_targetPrice TEXT,
    item_competitiveProduct TEXT,
    item_originalFinalPrice TEXT,
    item_originalMetaBriteItemPrice TEXT,
    item_deleted TEXT,
    item_priceAfterCoupon TEXT,
    item_metabriteCampaignId TEXT,
    receipts_id TEXT,
    FOREIGN KEY (receipts_id) REFERENCES receipts(receipts_id)
);
''')


cursor.execute('''
CREATE TABLE IF NOT EXISTS brands (
    brand_id TEXT PRIMARY KEY,
    barcode INTEGER,
    category TEXT,
    categoryCode TEXT,
    name TEXT,
    topBrand REAL,
    brandCode TEXT,
    cpg_id TEXT,
    FOREIGN KEY (cpg_id) REFERENCES cpg(cpg_id)
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS cpg (
    cpg_id TEXT PRIMARY KEY,
    cpg_ref TEXT
);
''')

# Committing the changes
conn.commit()

## Populating the tables

In [539]:
df_users.to_sql('users', conn, if_exists='replace', index=False)
df_receipts.to_sql('receipts', conn, if_exists='replace', index=False)
df_receipt_items_list.to_sql('receipt_items', conn, if_exists='replace', index=False)
df_brands.to_sql('brands', conn, if_exists='replace', index=False)
df_cpg.to_sql('cpg', conn, if_exists='replace', index=False)

## Reviewing Table Structures and Verifying Data Integrity

In [540]:
cursor.execute("SELECT COUNT(*) FROM receipts")
print("Receipts count:", cursor.fetchone()[0])

cursor.execute("SELECT COUNT(*) FROM receipt_items")
print("Receipt items count:", cursor.fetchone()[0])

cursor.execute("SELECT COUNT(*) FROM brands")
print("Brands count:", cursor.fetchone()[0])

Receipts count: 1119
Receipt items count: 6941
Brands count: 1167


### Printing first 5 rows in each table

In [541]:
tables = ['users', 'receipts', 'receipt_items', 'brands', 'cpg']

# Loop through each table and print the first 5 rows
for table in tables:
    print(f"Data from table: {table}")
    cursor.execute(f"SELECT * FROM {table} LIMIT 5;")
    
    # Fetch all rows returned by the SELECT statement
    rows = cursor.fetchall()
    
    # If rows are returned, print them
    if rows:
        for row in rows:
            print(row)
    else:
        print(f"No data found in table: {table}")
    
    # Print a separator for readability
    print("-" * 50)


Data from table: users
('5ff1e194b6a9d73a3a9f1052', 1, '2021-01-03 15:24:04.800000', '2021-01-03 15:25:37.858000', 'consumer', 'Email', 'WI')
('5ff1e1eacfcf6c399c274ae6', 1, '2021-01-03 15:25:30.554000', '2021-01-03 15:25:30.597000', 'consumer', 'Email', 'WI')
('5ff1e1e8cfcf6c399c274ad9', 1, '2021-01-03 15:25:28.354000', '2021-01-03 15:25:28.392000', 'consumer', 'Email', 'WI')
('5ff1e1b7cfcf6c399c274a5a', 1, '2021-01-03 15:24:39.626000', '2021-01-03 15:24:39.665000', 'consumer', 'Email', 'WI')
('5ff1e1f1cfcf6c399c274b0b', 1, '2021-01-03 15:25:37.564000', '2021-01-03 15:25:37.599000', 'consumer', 'Email', 'WI')
--------------------------------------------------
Data from table: receipts
('5ff1e1eb0a720f0523000575', 500.0, 'Receipt number 2 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)', '2021-01-03 15:25:31', '2021-01-03 15:25:31', '2021-01-03 15:25:31', '2021-01-03 15:25:36', '2021-01-03 15:25:31', 500.0, '2021-01-03 00:00:00', 5.0, 'FINISHED', 26.0, '5ff1e1eacfcf6

## SQL Queries for answering questions

### When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [542]:
unique_statuses = df_receipts['rewardsReceiptStatus'].unique()
print(unique_statuses)

['FINISHED' 'REJECTED' 'FLAGGED' 'SUBMITTED' 'PENDING']


**There is no ACCEPTED but logically FINISHED can be replaced with ACCEPTED**

In [543]:
# SQL query adjusted for the given table structures
sql_query = """
SELECT 
    rewardsReceiptStatus, 
    AVG(CAST(totalSpent AS REAL)) AS avgTotalSpent
FROM 
    receipts
WHERE 
    rewardsReceiptStatus IN ('REJECTED', 'FINISHED')
GROUP BY 
    rewardsReceiptStatus
ORDER BY 
    avgTotalSpent;
"""

# Execute the query
cursor.execute(sql_query)
results = cursor.fetchall()

print("Average Total Spent by Receipt Status:")
for result in results:
    print(f"Status: {result[0]}, Average Total Spent: {result[1]}")

Average Total Spent by Receipt Status:
Status: REJECTED, Average Total Spent: 23.326056338028184
Status: FINISHED, Average Total Spent: 80.85430501930502


### When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [544]:
# SQL query adjusted for the given table structures
sql_query = """
SELECT 
    rewardsReceiptStatus,
    SUM(purchasedItemCount) AS totalItemsPurchased
FROM 
    receipts
WHERE 
    rewardsReceiptStatus IN ('REJECTED', 'FINISHED')
GROUP BY 
    rewardsReceiptStatus;
"""

# Execute the query
cursor.execute(sql_query)
results = cursor.fetchall()

# Assuming the result could be in any order
finished_items = 0
rejected_items = 0
for status, total in results:
    if status == 'FINISHED':
        accepted_items = total
    elif status == 'REJECTED':
        rejected_items = total

# Compare and print the results
if accepted_items > rejected_items:
    print(f"FINISHED receipts have more items purchased: {accepted_items}")
elif rejected_items > accepted_items:
    print(f"REJECTED receipts have more items purchased: {rejected_items}")
else:
    print("The number of items purchased is equal for ACCEPTED and REJECTED receipts.")

FINISHED receipts have more items purchased: 8184.0


# Evaluating Data Quality Issues in the Data Provided

### In addition to addressing the issue of duplicates within the `users` table during the data wrangling phase, another significant concern identified is as follows:

In [545]:
# SQL query to count the total number of rows in 'receipt_items'
query_total_rows = "SELECT COUNT(*) FROM receipt_items;"

# Execute the query and fetch the result
cursor.execute(query_total_rows)
total_rows = cursor.fetchone()[0]

# SQL query to count missing values in 'item_needsFetchReview'
query_missing_needsFetchReview = """
SELECT COUNT(*) 
FROM receipt_items 
WHERE item_needsFetchReview IS NULL;
"""

# Execute the query and fetch the result for missing values
cursor.execute(query_missing_needsFetchReview)
missing_needsFetchReview_count = cursor.fetchone()[0]

# Calculate the percentage of missing values
percent_missing_needsFetchReview = (missing_needsFetchReview_count / total_rows) * 100
print(f"Percentage of missing values in 'item_needsFetchReview': {percent_missing_needsFetchReview:.2f}%")

Percentage of missing values in 'item_needsFetchReview': 88.29%


In [546]:
# SQL query to count missing values in 'item_needsFetchReviewReason'
query_missing_needsFetchReviewReason = """
SELECT COUNT(*) 
FROM receipt_items 
WHERE item_needsFetchReviewReason IS NULL;
"""

# Execute the query and fetch the result for missing values
cursor.execute(query_missing_needsFetchReviewReason)
missing_needsFetchReviewReason_count = cursor.fetchone()[0]

# Calculate the percentage of missing values
percent_missing_needsFetchReviewReason = (missing_needsFetchReviewReason_count / total_rows) * 100
print(f"Percentage of missing values in 'item_needsFetchReviewReason': {percent_missing_needsFetchReviewReason:.2f}%")

Percentage of missing values in 'item_needsFetchReviewReason': 96.84%


In [547]:
# Closing the database connection
conn.close()